In [ ]:
# Tokenizer for basic Python source code
import tokenize
from io import BytesIO

def tokenize_python_code(code):
    tokens = []
    reader = BytesIO(code.encode('utf-8')).readline
    for tok in tokenize.tokenize(reader):
        if tok.type != tokenize.ENCODING:
            tokens.append((tokenize.tok_name[tok.type], tok.string))
    return tokens



In [ ]:
# Simple parser using Python's built-in ast module
import ast

def parse_to_ast(code):
    try:
        return ast.parse(code)
    except SyntaxError as e:
        print("Syntax error while parsing:", e)
        return None



In [ ]:
# IR nodes to represent simplified language constructs

class IRNode:
    pass

class IRFunction(IRNode):
    def __init__(self, name, args, body):
        self.name = name
        self.args = args
        self.body = body

class IRAssignment(IRNode):
    def __init__(self, target, value):
        self.target = target
        self.value = value

class IRPrint(IRNode):
    def __init__(self, value):
        self.value = value

class IRInput(IRNode):
    def __init__(self, prompt=None, cast_type=None, target=None):
        self.prompt = prompt
        self.cast_type = cast_type  # e.g., 'int'
        self.target = target



In [ ]:
# Converts Python AST nodes into a simplified intermediate representation

def convert_ast_to_ir(py_ast):
    ir_nodes = []

    for node in py_ast.body:
        if isinstance(node, ast.FunctionDef):
            func_name = node.name
            func_args = [arg.arg for arg in node.args.args]
            body = []

            for stmt in node.body:
                if isinstance(stmt, ast.Assign):
                    target = stmt.targets[0].id
                    value = stmt.value

                    # Handle int(input("..."))
                    if isinstance(value, ast.Call):
                        if isinstance(value.func, ast.Name) and value.func.id == 'int':
                            inner = value.args[0]
                            if isinstance(inner, ast.Call) and inner.func.id == 'input':
                                prompt = inner.args[0].s if inner.args else None
                                body.append(IRInput(prompt=prompt, cast_type='int', target=target))
                                continue
                        elif value.func.id == 'input':
                            prompt = value.args[0].s if value.args else None
                            body.append(IRInput(prompt=prompt, cast_type='string', target=target))
                            continue

                    body.append(IRAssignment(target, ast.unparse(value)))

                elif isinstance(stmt, ast.Expr) and isinstance(stmt.value, ast.Call):
                    if isinstance(stmt.value.func, ast.Name) and stmt.value.func.id == 'print':
                        body.append(IRPrint(ast.unparse(stmt.value.args[0])))

            ir_nodes.append(IRFunction(func_name, func_args, body))

    return ir_nodes



In [ ]:
# Translates IR nodes into readable and functional Java code

def generate_java_code(ir_nodes):
    java_lines = ["import java.util.Scanner;", "", "public class TranslatedProgram {"]

    for func in ir_nodes:
        header = f"    public static void {func.name}("
        header += ", ".join([f"String {arg}" for arg in func.args])
        header += ") {"
        java_lines.append(header)

        java_lines.append("        Scanner sc = new Scanner(System.in);")

        for stmt in func.body:
            if isinstance(stmt, IRAssignment):
                java_lines.append(f"        var {stmt.target} = {stmt.value};")
            elif isinstance(stmt, IRPrint):
                java_lines.append(f"        System.out.println({stmt.value});")
            elif isinstance(stmt, IRInput):
                java_lines.append(f"        System.out.print(\"{stmt.prompt or ''}\");")
                if stmt.cast_type == 'int':
                    java_lines.append(f"        int {stmt.target} = Integer.parseInt(sc.nextLine());")
                else:
                    java_lines.append(f"        String {stmt.target} = sc.nextLine();")

        java_lines.append("    }")

    java_lines.append("}")
    return "\n".join(java_lines)



In [ ]:
# Runner for the full translation pipeline

sample_code = '''
def add():
    a = int(input("Enter first number: "))
    b = int(input("Enter second number: "))
    result = a + b
    print(result)
'''

# Step-by-step execution
tokens = tokenize_python_code(sample_code)
print("Tokens:\n", tokens)

py_ast = parse_to_ast(sample_code)
if py_ast:
    ir_nodes = convert_ast_to_ir(py_ast)
    print("\nIntermediate Representation:")
    for node in ir_nodes:
        print(f"Function: {node.name}, Args: {node.args}")

    print("\nGenerated Java Code:\n")
    java_code = generate_java_code(ir_nodes)
    print(java_code)

